## Import packages

In [ ]:
from kinetixsimulator.chemicalkinetics import KineticModel, Reaction, ReversibleReaction
from kinetixsimulator.guis import ProgressCurveGUI
from matplotlib import pyplot as plt
import numpy as np

## Construct a KineticModel Object and Set Initial Concentrations

Constructing a `KineticModel` object requires a numpy array of timepoints and a mixed-type list containing objects from the following container classes: `Reaction`, `ReversibleReaction`, and/or `MMReaction`. The `Reaction` and `ReversibleReaction` reaction classes organize data for irreversible and reversible reactions. The `MMReaction` class organizes data for enzymatic reactions that obey the single substrate Michaelis-Menten equation. A more detailed discussion on usage of this class is provded in [Technical Notes](#technical-notes). Constructing reaction objects requires a string representation of the reaction scheme as well as the names/values of rate constants. When generating reaction strings, keep the following formatting rules in mind:
1. Separate chemical species with a `+`
2. Separate substrates and products with a `->` or `<->`
3. Separate stoichiometric coefficients with a `*`
4. Separate species and characters with a space

Units of concentration and time can be designated by setting `concentration_units` and `time_units`, which default to µM and s.

By default, initial concentrations of all species are set to zero. Initial concentrations can be manually set using the `set_initial_concentrations` method provided within the `KineticModel` class. 

In [ ]:
# create a list of reaction objects
reactions = [
    ReversibleReaction(reaction_string='E + S <-> E:S', rconst_names=['kon', 'koff'], rconst_values=[1e2, 1]),
    Reaction(reaction_string='E:S -> E + P', rconst_name='kcat', rconst_value=5)
]

# create time array and construct the KineticModel object
time = np.linspace(0, 1000, 1000)
kinetic_model = KineticModel(time, reactions)

# set initial concentrations
kinetic_model.set_initial_concentration('E', 1e-3)
kinetic_model.set_initial_concentration('S', 2)

## Simulate and Plot

Reaction kinetics can be simulated using the `simulate` method.

In [ ]:
# simulate kinetics given rate constants and initial concentrations defined above
traces = kinetic_model.simulate(inplace=False)

# plot the results
fig, ax = plt.subplots()
for specie_name, trace in zip(kinetic_model.species, traces):
    ax.plot(kinetic_model.time, trace, label=specie_name)

ax.set_xlabel(f'Time ({kinetic_model.time_units})')
ax.set_ylabel(f'Concentration ({kinetic_model.concentration_units})')
ax.legend()

## Interactive Simulation

To launch the interactive simulation dashboard, construct a `ProgressCurveGUI` object and call the `launch` method. By default, sliders enabling control of rate constant values and specie initial concentrations will be rendered with a set range. You can prevent the display of particular sliders and override default slider ranges by passing a `slider_config` dictionary to the `launch` method. If you would like to hide traces for certain chemical species by providing a `hidden_species` list to the `launch` method.

Note that interactive simulation is only supported within the Jupyter environment (i.e. you can't run this outside of a Jupyter Notebook).

In [ ]:
gui = ProgressCurveGUI()
gui.launch(kinetic_model)

## Technical Notes

<a id="technical-notes"></a>

### Avoiding Un-Physical Solutions

If your model contains rate constants or initial concentrations that are exceptionally large, you may observe un-physical solutions (i.e. negative concentrations for some species). This can be avoided by lowering the values of the `integrator_atol` and `integrator_rtol` optional arguments in the `KineticModel` constructor, which set the error tolerance of the numerical integration algorithm used in the `simulate` method. Decreasing the increment between timepoints may help resolve these issues.

### Using the MMReaction Class

The `MMReaction` class provides a more convenient means to organize data for enzymatic reactions that obey the single substrate Michaelis-Menten equation. A `MMReaction` object can be constructed as follows:

```python
reaction = MMReaction(reaction_string='E + S <-> E:S -> E + P', Km_name='Km', Km_value=1, kcat_name='kcat', kcat_value=1)
```

The reaction strings used to construct a `MMReaction` object must contain an equilibrium describing the substrate binding step and an irreversible reaction corresponding to the chemical step. Currently, the class only supports single substrate Michaelis-Menten mechanisms. If your enzyme follows a multi substrate Michaelis-Menten mechanism, you might consider simulating your system under conditions enabling simplification to a single substrate mechanism. 

A word of caution: during construction of the `KineticModel` object, values for microscopic rate constants (i.e. substrate association and dissociation rate constants) that are consistent with the provided steady-state constants will be selected arbitrarily. Keep this in mind when interpreting simulation results! 